In [2]:
# 문제 1
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import re
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# API Key 설정
open_api_key = os.getenv("OPEN_API_KEY")

# Neo4j 연결 정보
graph = Neo4jGraph(
    url="bolt://3.38.163.190:7687", username="neo4j", password="kstr1234"
)
llm = ChatOpenAI(temperature=0, model="gpt-4o")


# 문제 풀이 함수 (LLM이 주요 개념도 포함하여 반환)
def solve_problem_with_llm(problem_text):
    prompt = f"""
    다음 초등 수학 문제를 풀고, 풀이에 사용된 주요 개념을 추출하세요:

    문제:
    {problem_text}

    출력 형식:
    1. 풀이:
    [여기에 상세한 풀이를 작성하세요]

    2. 주요 개념:
    [풀이에 사용된 주요 초등 수학적 개념을 목록으로 작성하세요]
    """
    result = llm(prompt).content
    if "2. 주요 개념:" in result:
        solution, key_concepts = result.split("2. 주요 개념:")
        key_concepts = re.findall(r"-\s(.*?\.)", key_concepts, re.DOTALL)
        return solution.strip(), key_concepts
    else:
        return "추가 정보가 필요합니다.", []


# 풀이 불가능한 경우 예상 주요 개념 추출 함수
def extract_concepts_from_context(problem_text):
    context_prompt = f"""
    다음 문제를 분석하고, 이를 해결하는 데 필요할 것으로 예상되는 주요 초등 수학적 개념을 추출하세요:

    문제:
    {problem_text}

    출력:
    [문제를 해결하기 위해 필요한 주요 초등 수학적 개념 목록]
    """
    result = llm(context_prompt).content
    return [concept.strip() for concept in result.split(",") if concept.strip()]


# LLM 응답에서 유사도 점수 추출 함수
def extract_similarity_score(response):
    try:
        match = re.search(r"(\d+(\.\d+)?)", response)
        if match:
            return float(match.group(1))  # 첫 번째 숫자 반환
    except ValueError:
        pass
    return 0.0


# 문제 분류 함수
# 각 주요 개념(key_concepts)에 대해 분류를 수행하여 대분류-중분류-하위분류-최하위분류를 추출합니다.
def classify_problem(key_concepts):
    all_classifications = []

    for key_concept in key_concepts:
        classifications = []

        # 단계 1: 모든 Category 가져오기 및 문맥적 유사도 계산
        category_query = """
        MATCH (c:Category)
        RETURN c.name AS category
        """
        category_results = graph.query(category_query)

        if not category_results:
            continue

        best_category = max(
            category_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['category']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_category:
            continue

        best_category_name = best_category["category"]

        # 단계 2~4: Subcategory, Branch, Leaf도 동일한 방식으로 처리
        subcategory_query = f"""
        MATCH (c:Category)-[:HAS_CHILD]->(s:Subcategory)
        WHERE c.name = '{best_category_name}'
        RETURN s.name AS subcategory
        """
        subcategory_results = graph.query(subcategory_query)

        if not subcategory_results:
            continue

        best_subcategory = max(
            subcategory_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['subcategory']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_subcategory:
            continue

        best_subcategory_name = best_subcategory["subcategory"]

        branch_query = f"""
        MATCH (s:Subcategory)-[:HAS_CHILD]->(b:Branch)
        WHERE s.name = '{best_subcategory_name}'
        RETURN b.name AS branch
        """
        branch_results = graph.query(branch_query)

        if not branch_results:
            continue

        best_branch = max(
            branch_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['branch']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_branch:
            continue

        best_branch_name = best_branch["branch"]

        leaf_query = f"""
        MATCH (b:Branch)-[:HAS_CHILD]->(l:Leaf)
        WHERE b.name = '{best_branch_name}'
        RETURN l.name AS leaf
        """
        leaf_results = graph.query(leaf_query)

        if not leaf_results:
            continue

        best_leaf = max(
            leaf_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['leaf']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_leaf:
            continue

        best_leaf_name = best_leaf["leaf"]

        classifications.append(
            {
                "c.name": best_category_name,
                "s.name": best_subcategory_name,
                "b.name": best_branch_name,
                "l.name": best_leaf_name,
            }
        )

        unique_classifications = []
        seen_combinations = set()

        for item in classifications:
            combination = (
                item["c.name"],
                item["s.name"],
                item["b.name"],
                item["l.name"],
            )
            if combination not in seen_combinations:
                seen_combinations.add(combination)
                unique_classifications.append(item)

        # 주요 개념마다 결과를 추가합니다.
        all_classifications.extend(unique_classifications[:2])

    return all_classifications


# 문제 텍스트
problem_text = "원기둥과 구를 사용하여 만든 모양입니다. 만든 모양을 앞에서 본 모양의 넓이는 몇 $cm^2$인지 구해 보세요. (원주율 : $3$)"

# 문제 풀이 및 주요 개념 추출
solution, key_concepts = solve_problem_with_llm(problem_text)

# 주요 개념이 비어 있을 경우, 예상 주요 개념 추출
if isinstance(key_concepts, list) and not key_concepts:
    solution = "추가 정보가 필요합니다."
    key_concepts = extract_concepts_from_context(problem_text)

# 주요 개념에 대한 대분류-중분류-하위분류-최하위분류 생성
full_context = []
if key_concepts:
    full_context = classify_problem(key_concepts)

# 분류 데이터를 정리
processed_context = [
    f"- 대분류: {item.get('c.name', '-')}, 중분류: {item.get('s.name', '-')}, 하위분류: {item.get('b.name', '-')}, 최하위분류: {item.get('l.name', '-')}"
    for item in full_context
]

# 출력 결과 생성
output = f"""
1. 문제:
{problem_text}

2. 풀이 과정:
{solution}

3. 주요 개념:
{', '.join(key_concepts) if key_concepts else '- 없음'}

4. 대분류-중분류-하위분류-최하위분류:
"""

for context in enumerate(processed_context):
    output += f"{context}\n"

print(output)


C:\Users\boar2\AppData\Local\Temp\ipykernel_19404\3315972182.py:38: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(prompt).content



1. 문제:
원기둥과 구를 사용하여 만든 모양입니다. 만든 모양을 앞에서 본 모양의 넓이는 몇 $cm^2$인지 구해 보세요. (원주율 : $3$)

2. 풀이 과정:
추가 정보가 필요합니다.

3. 주요 개념:
문제를 해결하기 위해 필요한 주요 초등 수학적 개념 목록:

1. **기하학적 도형의 이해**: 원기둥과 구의 기본적인 형태와 특성을 이해하는 것이 필요합니다.

2. **넓이 계산**: 도형의 넓이를 계산하는 방법을 알아야 합니다. 특히, 원의 넓이 계산이 필요합니다.

3. **원주율의 이해**: 원주율(π)을 사용하여 원의 넓이를 계산하는 방법을 이해해야 합니다. 문제에서는 원주율을 3으로 사용하라고 주어졌습니다.

4. **단위 변환**: 넓이를 구할 때 사용하는 단위(예: cm^2)에 대한 이해가 필요합니다.

5. **공간 시각화 능력**: 3차원 도형을 2차원에서 바라보았을 때의 모양을 상상하고 이해하는 능력이 필요합니다.

6. **덧셈과 뺄셈**: 여러 도형의 넓이를 합산하거나 필요에 따라 특정 부분의 넓이를 제외하는 계산이 필요할 수 있습니다.

4. 대분류-중분류-하위분류-최하위분류:
(0, '- 대분류: 수와 연산, 중분류: 분수, 하위분류: 분수의 개념 이해, 최하위분류: 분모와 분자의 의미')



In [3]:
# 문제2
from langchain.prompts import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import re
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# API Key 설정
open_api_key = os.getenv("OPEN_API_KEY")

# Neo4j 연결 정보
graph = Neo4jGraph(
    url="bolt://3.38.163.190:7687", username="neo4j", password="kstr1234"
)
llm = ChatOpenAI(temperature=0, model="gpt-4o")


# 문제 풀이 함수 (LLM이 주요 개념도 포함하여 반환)
def solve_problem_with_llm(problem_text):
    prompt = f"""
    다음 초등 수학 문제를 풀고, 풀이에 사용된 주요 개념을 추출하세요:

    문제:
    {problem_text}

    출력 형식:
    1. 풀이:
    [여기에 상세한 풀이를 작성하세요]

    2. 주요 개념:
    [풀이에 사용된 주요 초등 수학적 개념을 목록으로 작성하세요]
    """
    result = llm(prompt).content
    if "2. 주요 개념:" in result:
        solution, key_concepts = result.split("2. 주요 개념:")
        key_concepts = re.findall(r"-\s(.*?\.)", key_concepts, re.DOTALL)
        return solution.strip(), key_concepts
    else:
        return "추가 정보가 필요합니다.", []


# 풀이 불가능한 경우 예상 주요 개념 추출 함수
def extract_concepts_from_context(problem_text):
    context_prompt = f"""
    다음 문제를 분석하고, 이를 해결하는 데 필요할 것으로 예상되는 주요 초등 수학적 개념을 추출하세요:

    문제:
    {problem_text}

    출력:
    [문제를 해결하기 위해 필요한 주요 초등 수수학적 개념 목록]
    """
    result = llm(context_prompt).content
    return [concept.strip() for concept in result.split(",") if concept.strip()]


# LLM 응답에서 유사도 점수 추출 함수
def extract_similarity_score(response):
    try:
        match = re.search(r"(\d+(\.\d+)?)", response)
        if match:
            return float(match.group(1))  # 첫 번째 숫자 반환
    except ValueError:
        pass
    return 0.0


# 문제 분류 함수
# 각 주요 개념(key_concepts)에 대해 분류를 수행하여 대분류-중분류-하위분류-최하위분류를 추출합니다.
def classify_problem(key_concepts):
    all_classifications = []

    for key_concept in key_concepts:
        classifications = []

        # 단계 1: 모든 Category 가져오기 및 문맥적 유사도 계산
        category_query = """
        MATCH (c:Category)
        RETURN c.name AS category
        """
        category_results = graph.query(category_query)

        if not category_results:
            continue

        best_category = max(
            category_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['category']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_category:
            continue

        best_category_name = best_category["category"]

        # 단계 2~4: Subcategory, Branch, Leaf도 동일한 방식으로 처리
        subcategory_query = f"""
        MATCH (c:Category)-[:HAS_CHILD]->(s:Subcategory)
        WHERE c.name = '{best_category_name}'
        RETURN s.name AS subcategory
        """
        subcategory_results = graph.query(subcategory_query)

        if not subcategory_results:
            continue

        best_subcategory = max(
            subcategory_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['subcategory']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_subcategory:
            continue

        best_subcategory_name = best_subcategory["subcategory"]

        branch_query = f"""
        MATCH (s:Subcategory)-[:HAS_CHILD]->(b:Branch)
        WHERE s.name = '{best_subcategory_name}'
        RETURN b.name AS branch
        """
        branch_results = graph.query(branch_query)

        if not branch_results:
            continue

        best_branch = max(
            branch_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['branch']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_branch:
            continue

        best_branch_name = best_branch["branch"]

        leaf_query = f"""
        MATCH (b:Branch)-[:HAS_CHILD]->(l:Leaf)
        WHERE b.name = '{best_branch_name}'
        RETURN l.name AS leaf
        """
        leaf_results = graph.query(leaf_query)

        if not leaf_results:
            continue

        best_leaf = max(
            leaf_results,
            key=lambda x: extract_similarity_score(
                llm(
                    f"'{key_concept}'와(과) '{x['leaf']}'의 유사도를 0과 1 사이로 평가하세요."
                ).content
            ),
            default=None,
        )

        if not best_leaf:
            continue

        best_leaf_name = best_leaf["leaf"]

        classifications.append(
            {
                "c.name": best_category_name,
                "s.name": best_subcategory_name,
                "b.name": best_branch_name,
                "l.name": best_leaf_name,
            }
        )

        unique_classifications = []
        seen_combinations = set()

        for item in classifications:
            combination = (
                item["c.name"],
                item["s.name"],
                item["b.name"],
                item["l.name"],
            )
            if combination not in seen_combinations:
                seen_combinations.add(combination)
                unique_classifications.append(item)

        # 주요 개념마다 결과를 추가합니다.
        all_classifications.extend(unique_classifications[:2])

    return all_classifications


# 문제 텍스트
problem_text = "아람이는 $5000$ 원짜리 샌드위치 $1$ 개와 $6100$ 원짜리 샐러드 $1$ 개를 사려고 합니다. 아람이가 $7200$ 원을 가지고 있을 때, 샌드위치 $1$ 개와 샐러드 $1$ 개를 사려면 얼마 더 필요한지 구해 보세요."

# 문제 풀이 및 주요 개념 추출
solution, key_concepts = solve_problem_with_llm(problem_text)

# 주요 개념이 비어 있을 경우, 예상 주요 개념 추출
if isinstance(key_concepts, list) and not key_concepts:
    solution = "추가 정보가 필요합니다."
    key_concepts = extract_concepts_from_context(problem_text)

# 주요 개념에 대한 대분류-중분류-하위분류-최하위분류 생성
full_context = []
if key_concepts:
    full_context = classify_problem(key_concepts)

# 분류 데이터를 정리
processed_context = [
    f"- 대분류: {item.get('c.name', '-')}, 중분류: {item.get('s.name', '-')}, 하위분류: {item.get('b.name', '-')}, 최하위분류: {item.get('l.name', '-')}"
    for item in full_context
]

# 출력 결과 생성
output = f"""
1. 문제:
{problem_text}

2. 풀이 과정:
{solution}

3. 주요 개념:
{', '.join(key_concepts) if key_concepts else '- 없음'}

4. 대분류-중분류-하위분류-최하위분류:
"""

for context in enumerate(processed_context):
    output += f"{context}\n"

print(output)


1. 문제:
아람이는 $5000$ 원짜리 샌드위치 $1$ 개와 $6100$ 원짜리 샐러드 $1$ 개를 사려고 합니다. 아람이가 $7200$ 원을 가지고 있을 때, 샌드위치 $1$ 개와 샐러드 $1$ 개를 사려면 얼마 더 필요한지 구해 보세요.

2. 풀이 과정:
1. 풀이:
   아람이가 사려고 하는 샌드위치와 샐러드의 총 가격을 먼저 계산합니다. 샌드위치의 가격은 5000원이고, 샐러드의 가격은 6100원이므로, 두 개의 총 가격은 다음과 같습니다.

   \[
   5000 + 6100 = 11100 \text{원}
   \]

   아람이가 가지고 있는 돈은 7200원이므로, 필요한 금액을 계산하기 위해 가지고 있는 돈을 총 가격에서 뺍니다.

   \[
   11100 - 7200 = 3900 \text{원}
   \]

   따라서, 아람이는 샌드위치 1개와 샐러드 1개를 사기 위해 3900원이 더 필요합니다.

3. 주요 개념:
덧셈: 두 물건의 가격을 합산하여 총 가격을 계산하는 과정에서 사용되었습니다., 뺄셈: 가지고 있는 돈에서 총 가격을 빼서 추가로 필요한 금액을 계산하는 과정에서 사용되었습니다., 비교: 가지고 있는 돈과 필요한 총 금액을 비교하여 부족한 금액을 찾는 과정에서 사용되었습니다.

4. 대분류-중분류-하위분류-최하위분류:
(0, '- 대분류: 수와 연산, 중분류: 혼합 계산, 하위분류: 혼합 계산의 사례, 최하위분류: 네 연산이 섞인 계산')
(1, '- 대분류: 변화와 관계, 중분류: 등호와 동치 관계, 하위분류: 등호 사용하기, 최하위분류: 크기가 같은 두 양의 관계를 식으로 나타내기')
(2, '- 대분류: 수와 연산, 중분류: 소수, 하위분류: 소수의 크기 비교, 최하위분류: 구체물이나 그림을 활용한 크기 비교')

